In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob
import pickle
import matplotlib.pyplot as plt
import joblib

import random
import os

import cv2
from PIL import Image

This notebook is pretty messy and I wouldn't normally share it, but someone asked about how I created what I called "threshold images" in my notebook (https://www.kaggle.com/therocket290/indoor-navigation-push-to-hallway-post-process?scriptVersionId=57642694). I don't have time to edit it, so I am just sharing this as is. 

I wanted to create images of each floor that made it easy to push predicted locations to the hallways. I used cv2.threshold from OpenCV to create these images. The hallways are white and the rest is black, but I had trouble making the outside of the building black as well. You can see my failed attempt at drawing a countour around the entire building. I wish I had more time to work on it, but I am currently too busy with work and family.

In [ ]:
def view(array):
    img = Image.fromarray(array)
    img.show()

In [ ]:
meta_dir = '../input/indoor-location-navigation/metadata'
building_paths = os.listdir(meta_dir)

In [ ]:
building_paths = []
for path in os.listdir('../input/wifi-features-all'):
    if (len(path.split('_')[0])>20) and (not path.split('_')[0] in building_paths):
        building_paths.append(path.split('_')[0])

In [ ]:
building_paths = building_paths[0:1]

gray_list = []
thresh_list = []
thresh_list2 = []
mask_list = []
floor_list = []
blur_list = []
combine_list = []
#building_list = []
contours_list = []


light_white = (0, 0, 200)
dark_white = (100, 100, 250)
for building in building_paths:
    grays = []
    threshes = []
    threshes2 = []
    masks = []
    floors_in_building = []
    blurs = []
    combines = []
    #buildings = []
    floor_contours = []
    
    #buildings.append(building)
    floors = os.listdir(os.path.join(meta_dir,building))
    for floor in floors:
        image_path = glob.glob(os.path.join(meta_dir,building,floor,'*.png'))
        img = Image.open(image_path[0]).convert('LA')
        gray = np.asarray(img)
        blur = cv2.GaussianBlur(gray,(5,5),500)
        im = cv2.imread(image_path[0], cv2.IMREAD_UNCHANGED) #cv2.IMREAD_UNCHANGED
        #im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        #gray = cv2.cvtColor(im,cv2.COLOR_RGB2GRAY)
        flag, thresh = cv2.threshold(gray, 245, 255, cv2.THRESH_BINARY)
        flag2, thresh2 = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
        #contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        #contours = sorted(contours, key=cv2.contourArea,reverse=True)
        hsv_im = cv2.cvtColor(im, cv2.COLOR_RGB2HSV)
        mask_white = cv2.inRange(hsv_im, light_white, dark_white)
        result = cv2.bitwise_and(im, im, mask=mask_white)
        grays.append(gray)
        threshes.append(thresh)
        #masks.append(mask_white)
        floors_in_building.append(floor)
        #blurs.append(blur)
        threshes2.append(thresh2)
        combine = np.maximum(thresh[:,:,1],thresh2[:,:,1])
        combines.append(combine)
        
        contours, hierarchy = cv2.findContours(thresh[:,:,1],cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea,reverse=True)
        floor_contours.append(contours)
    
    gray_list.append(grays)
    thresh_list.append(threshes)
    thresh_list2.append(threshes2)
    combine_list.append(combines)
    #mask_list.append(masks)
    floor_list.append(floors_in_building)
    #blur_list.append(blurs)
    contours_list.append(floor_contours)
    joblib.dump(threshes, 'thresh_list_'+str(building)+'.pkl')
    joblib.dump(floor_list, 'floor_list_'+str(building)+'.pkl')

In [ ]:
(thresh_list[0][0] != thresh_list2[0][0]).sum()

In [ ]:
#combine_ex = np.maximum(thresh_list[0][0][:,:,1], thresh_list2[0][0][:,:,1])
combine_ex = np.maximum(thresh_list[0][0], thresh_list2[0][0])

In [ ]:
img = Image.fromarray(combine_ex)
plt.imshow(img)

In [ ]:
img = Image.fromarray(thresh_list2[0][0])
plt.imshow(img)

In [ ]:
img = Image.fromarray(thresh_list[0][0])
plt.imshow(img)

In [ ]:
im = cv2.imread(image_path[0], cv2.IMREAD_UNCHANGED)
cnt = contours[0]
test_cnt = cv2.drawContours(im, [cnt], 0, (0,255,0), 3)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
plt.imshow(test_cnt)

In [ ]:
#fig, axs = plt.subplots(12, 2)
total_floors = []
for k, building in enumerate(building_paths):
    num_floors = len(floor_list[k])
    for e, floor in enumerate(floor_list[k]):
        #print('floors: ', num_floors)
        total_floors.append(1)
        fig, ax = plt.subplots(1,4,figsize=(12, 12))
        plt.subplot(1,4,1)
        plt.title(building+' '+floor)
        array = thresh_list[k][e]
        img = Image.fromarray(array)
        plt.imshow(img)
        plt.subplot(1,4,2)
        array2 = gray_list[k][e]
        img2 = Image.fromarray(array2)
        plt.imshow(img2)
        plt.subplot(1,4,3)
        array3 = thresh_list2[k][e]
        img3 = Image.fromarray(array3)
        plt.imshow(img3)
        plt.subplot(1,4,4)
        array3 = combine_list[k][e]
        img3 = Image.fromarray(array3)
        plt.imshow(img3)